In [1]:
from _plotly_future_ import v4_subplots

In [2]:
import pandas as pd
import numpy as np

In [3]:
from dgutils.pandas import add_column

/Users/alicegao/anaconda2/envs/dg_work_old_tf/lib/python2.7/site-packages/tqdm/autonotebook.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [4]:
from collections import Counter

In [5]:
from utils import EvalMetric

Using TensorFlow backend.
/Users/alicegao/anaconda2/envs/dg_work_old_tf/lib/python2.7/site-packages/tensorflow/python/framework/tensor_util.py:32: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  from tensorflow.python.framework import fast_tensor_util
/Users/alicegao/anaconda2/envs/dg_work_old_tf/lib/python2.7/site-packages/scipy/sparse/lil.py:19: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  from . import _csparsetools
/Users/alicegao/anaconda2/envs/dg_work_old_tf/lib/python2.7/site-packages/scipy/sparse/csgraph/__init__.py:182: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  from ._shortest_path import shortest_path, floyd_warshall, dijkstra,\
/Users/alicegao/anaconda2/envs/dg_work_old_tf/lib/python2.7/site-packages/scipy/sparse/csgraph/_validation.py:5: RuntimeWar

In [6]:
import matplotlib.pyplot as plt 
# import seaborn as sns
import pandas as pd
# sns.set(color_codes=True)
import cufflinks as cf
cf.go_offline()
cf.set_config_file(theme='ggplot')

/Users/alicegao/anaconda2/envs/dg_work_old_tf/lib/python2.7/site-packages/scipy/spatial/__init__.py:99: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject



In [7]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.tools as tls
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import plotly.figure_factory as ff
init_notebook_mode(connected=True)

In [8]:
result_filenmae = 'result/prediction.rfam.2019_09_25_1.sample100.pkl'

In [9]:
df = pd.read_pickle(result_filenmae)


In [10]:
# tmp load rnafold & clean data TODO remove once rerun prediction
df_tmp = pd.read_pickle('result/rfam_rnafold_sample_100.pkl.gz')
df_tmp = df_tmp[['seq_id', 'rnafold_one_idx']]
df = pd.merge(df, df_tmp, on='seq_id', how='inner')
df = df.rename(columns={'one_idx': 'truth_one_idx', 'rnafold_one_idx': 'one_idx'})
# tmp fix
df = add_column(df, 'pred_idx', ['pred_idx'], lambda l: [item for sublist in l for item in sublist])

In [11]:
df.head()

,len,one_idx,pred_idx,seq,seq_id,truth_one_idx
0,117,"[([1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 13, 14, 15,...","[([0, 1, 2, 3, 4, 5, 6, 7, 8, 28, 30, 31, 32, ...",UUCGGUGGUUAUAGCGGUGGGGAAACACCCGGUCCCAUUCCGAACC...,RF00001_A,"([0, 1, 2, 3, 4, 5, 6, 7, 8, 13, 14, 16, 17, 1..."
1,150,"[([3, 4, 5, 6, 7, 9, 10, 11, 12, 16, 17, 18, 1...","[([3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 16, 17, 1...",UAGCUUCAGCGAUGGAUCGGUUGCAUCGAGUAUCGAUGAAGAACGC...,RF00002_B,"([32, 33, 35, 36, 43, 44, 45, 46, 47, 58, 59, ..."
2,161,"[([2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16...","[([2, 3, 4, 15, 16, 17, 18, 19, 20, 22, 23, 24...",AUACUUACCUGGACGGGGUCAAUGGAUGAUCUAUAAGGUCCAUGGC...,RF00003_B,"([11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, ..."
3,145,"[([2, 3, 4, 5, 6, 10, 11, 12, 13, 27, 28, 29, ...","[([10, 11, 12, 13, 29, 30, 31, 32, 49, 50, 51,...",AUAUCUUCUCGGCUAUUUAGCUAAGAUCAAGUUAUUAAACUGUUCU...,RF00004_A,"([6, 7, 8, 10, 11, 12, 13, 46, 47, 48, 49, 50,..."
4,72,"[([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1...","[([0, 1, 2, 3, 4, 5, 6, 14, 16, 17, 18, 19, 25...",GGGCCCAUAGCUCAGUGGUAGAGUGCCUCCUUUGCAAGGAGGAUGC...,RF00005_A,"([0, 1, 2, 3, 4, 5, 6, 9, 10, 11, 12, 25, 26, ..."


In [29]:
df[['len']].iplot(kind='histogram', bins=100)

In [12]:
# convert to tuple so we can use set()
def _to_tuple(idxes):
    x = []
    for _a in idxes:
        a = []
        for i, j in zip(_a[0], _a[1]):
            a.append((i, j))
        if len(a) > 0:
            a = tuple(a)
            x.append(a)
    return x

In [13]:
def _to_mat(x, l):
    # x is tuple of tuple (2 elements)
    # convert to list of 2 lists
    y = [[a[0] for a in x], [a[1] for a in x]]
    z = np.zeros((l, l))
    z[y] = 1
    return z

In [14]:
n_no_struct = 0
df_metric = []

for _, row in df.iterrows():
    seq = row['seq']
    rnafold_struct_count = Counter(_to_tuple(row['one_idx']))
    model_struct_count = Counter(_to_tuple(row['pred_idx']))
#     # only keep structures that occurs more than 2 times (2% in sampled structures)
#     rnafold_struct_count = {k: v for k, v in rnafold_struct_count.iteritems() if v >=2}
#     model_struct_count = {k: v for k, v in model_struct_count.iteritems() if v >=2}
#     # only keep structures that occurs more than 5 times (5% in sampled structures)
#     rnafold_struct_count = {k: v for k, v in rnafold_struct_count.iteritems() if v >=5}
#     model_struct_count = {k: v for k, v in model_struct_count.iteritems() if v >=5}
#     # only keep structures that occurs more than 10 times (10% in sampled structures)
#     rnafold_struct_count = {k: v for k, v in rnafold_struct_count.iteritems() if v >=10}
#     model_struct_count = {k: v for k, v in model_struct_count.iteritems() if v >=10}
    
    # skip the case if RNAfold outputs no structure
    if len(rnafold_struct_count) == 0:
        n_no_struct += 1
        continue
    
    # for each structure from RNAfold
    # evaluate against all structures from the model, pick the closest one, record performance
#     result = []
    for rnafold_struct, rnafold_count in rnafold_struct_count.iteritems():
        r_s = _to_mat(rnafold_struct, len(seq))
        sensitivity = 0
        ppv = 0
        f_measure = 0
        best_m_s = None
        for model_struct in model_struct_count.keys():
            m_s = _to_mat(model_struct, len(seq))
            _sensitivity = EvalMetric.sensitivity(m_s, r_s)
            _ppv = EvalMetric.ppv(m_s, r_s)
            _f_measure = EvalMetric.f_measure(_sensitivity, _ppv)
            if _f_measure > f_measure:
                sensitivity = _sensitivity + 0  # copy
                ppv = _ppv + 0
                f_measure = _f_measure + 0
                best_m_s = list(model_struct)
#         result.append((sensitivity, ppv, f_measure))
        df_metric.append({
            'seq': seq,
            'rnafold_idx': list(rnafold_struct),
            'closest_model_idx': best_m_s,
            'sensitivity': sensitivity,
            'ppv': ppv,
            'f_measure': f_measure,
            'rnafold_count': rnafold_count,
        })
    
#     print rnafold_struct_count
#     print model_struct_count
#     print result
#     print ''
    
    
print n_no_struct
df_metric = pd.DataFrame(df_metric)


/Users/alicegao/anaconda2/envs/dg_work_old_tf/lib/python2.7/site-packages/ipykernel_launcher.py:6: FutureWarning:

Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.

utils.py:325: RuntimeWarning:

invalid value encountered in double_scalars



0


In [32]:
# TODO add seq_id in above code
df_metric = pd.merge(df_metric, df[['seq', 'len']], on='seq', how='left')

In [33]:
df_metric.head()

,closest_model_idx,f_measure,ppv,rnafold_count,rnafold_idx,sensitivity,seq,len
0,"[(0, 115), (1, 114), (2, 113), (3, 112), (4, 1...",0.904110,0.942857,1,"[(1, 114), (2, 113), (3, 112), (4, 111), (5, 1...",0.868421,UUCGGUGGUUAUAGCGGUGGGGAAACACCCGGUCCCAUUCCGAACC...,117
1,"[(0, 115), (1, 114), (2, 113), (3, 112), (4, 1...",0.833333,0.882353,1,"[(0, 115), (1, 114), (2, 113), (3, 112), (4, 1...",0.789474,UUCGGUGGUUAUAGCGGUGGGGAAACACCCGGUCCCAUUCCGAACC...,117
2,"[(0, 115), (1, 114), (2, 113), (3, 112), (4, 1...",0.800000,0.896552,2,"[(2, 113), (3, 112), (4, 111), (5, 110), (6, 1...",0.722222,UUCGGUGGUUAUAGCGGUGGGGAAACACCCGGUCCCAUUCCGAACC...,117
3,"[(0, 115), (1, 114), (2, 113), (3, 112), (4, 1...",0.864865,0.941176,1,"[(1, 114), (2, 113), (3, 112), (4, 111), (5, 1...",0.800000,UUCGGUGGUUAUAGCGGUGGGGAAACACCCGGUCCCAUUCCGAACC...,117
4,"[(0, 115), (1, 114), (2, 113), (3, 112), (4, 1...",0.885714,0.911765,1,"[(1, 114), (2, 113), (3, 112), (4, 111), (5, 1...",0.861111,UUCGGUGGUUAUAGCGGUGGGGAAACACCCGGUCCCAUUCCGAACC...,117


In [30]:
fig = make_subplots(rows=1, cols=2, subplot_titles=['Sensitivity', 'PPV'])

for col_idx, metric_name in enumerate(['sensitivity', 'ppv']):

    col = col_idx + 1
    fig.add_trace(go.Histogram(x=df_metric[df_metric['rnafold_count'] == 1][metric_name], 
                              nbinsx=20, histnorm='probability', 
                               name='1 ({})'.format(len(df_metric[df_metric['rnafold_count'] == 1])),
                              marker_color='#ffffcc', showlegend=True if col_idx == 0 else False), 
                 row=1, col=col)
    fig.add_trace(go.Histogram(x=df_metric[df_metric['rnafold_count'] == 2][metric_name], 
                              nbinsx=20, histnorm='probability', 
                               name='2 ({})'.format(len(df_metric[df_metric['rnafold_count'] == 2])),
                              marker_color='#c7e9b4', showlegend=True if col_idx == 0 else False),
                 row=1, col=col)
    fig.add_trace(go.Histogram(x=df_metric[df_metric['rnafold_count'] == 3][metric_name], 
                              nbinsx=20, histnorm='probability', 
                               name='3 ({})'.format(len(df_metric[df_metric['rnafold_count'] == 3])),
                              marker_color='#7fcdbb', showlegend=True if col_idx == 0 else False),
                 row=1, col=col)
    fig.add_trace(go.Histogram(x=df_metric[df_metric['rnafold_count'] == 4][metric_name], 
                              nbinsx=20, histnorm='probability', 
                               name='4 ({})'.format(len(df_metric[df_metric['rnafold_count'] == 4])),
                              marker_color='#41b6c4', showlegend=True if col_idx == 0 else False),
                 row=1, col=col)
    fig.add_trace(go.Histogram(x=df_metric[df_metric['rnafold_count'] == 5][metric_name], 
                              nbinsx=20, histnorm='probability', 
                               name='5 ({})'.format(len(df_metric[df_metric['rnafold_count'] == 5])),
                              marker_color='#1d91c0', showlegend=True if col_idx == 0 else False),
                 row=1, col=col)
    fig.add_trace(go.Histogram(x=df_metric[(df_metric['rnafold_count'] > 5) & (df_metric['rnafold_count'] <= 10)]['sensitivity'], 
                              nbinsx=20, histnorm='probability', 
                               name='(5, 10] ({})'.format(len(df_metric[(df_metric['rnafold_count'] > 5) & (df_metric['rnafold_count'] <= 10)])),
                              marker_color='#225ea8', showlegend=True if col_idx == 0 else False),
                 row=1, col=col)
    fig.add_trace(go.Histogram(x=df_metric[df_metric['rnafold_count'] > 10][metric_name], 
                              nbinsx=20, histnorm='probability', 
                               name='>10 ({})'.format(len(df_metric[df_metric['rnafold_count'] > 10])),
                              marker_color='#0c2c84', showlegend=True if col_idx == 0 else False), 
                 row=1, col=col)

fig = fig.update(layout=dict(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
#     xaxis=dict(showgrid=False, zeroline=False,
#                tickfont=dict(size=18, color='#000'),
#                titlefont=dict(size=24, color='#000'),
#                showline=True, mirror=True),
#     yaxis=dict(showgrid=True, gridwidth=1, gridcolor='darkgray',
#                tickfont=dict(size=18, color='#000'),
#                titlefont=dict(size=24, color='#000'),
#                title='Percentage', showline=True, mirror=True),
    title=result_filenmae,
#     title=metric_name,
#     titlefont=dict(size=24, color='#000'),
#     legend=dict(orientation="h",
#                 font=dict(
# #             family='sans-serif',
#             size=24,
#             color='#000'
#         ),),
))
iplot(fig)

In [17]:
# for metric_name in ['sensitivity', 'ppv']:
#     fig = go.Figure()
#     # fig = df_metric[df_metric['rnafold_count'] > 5][['sensitivity', 'ppv']].iplot(kind='histogram', bins=20,
#     #                                                     histnorm='probability', opacity=1,
#     #                                                      barmode='group', asFigure=True)

#     fig.add_trace(go.Histogram(x=df_metric[df_metric['rnafold_count'] == 1][metric_name], 
#                               nbinsx=20, histnorm='probability', name='1'))
#     fig.add_trace(go.Histogram(x=df_metric[df_metric['rnafold_count'] == 2][metric_name], 
#                               nbinsx=20, histnorm='probability', name='2'))
#     fig.add_trace(go.Histogram(x=df_metric[df_metric['rnafold_count'] == 3][metric_name], 
#                               nbinsx=20, histnorm='probability', name='3'))
#     fig.add_trace(go.Histogram(x=df_metric[df_metric['rnafold_count'] == 4][metric_name], 
#                               nbinsx=20, histnorm='probability', name='4'))
#     fig.add_trace(go.Histogram(x=df_metric[df_metric['rnafold_count'] == 5][metric_name], 
#                               nbinsx=20, histnorm='probability', name='5'))
#     fig.add_trace(go.Histogram(x=df_metric[(df_metric['rnafold_count'] > 5) & (df_metric['rnafold_count'] <= 10)]['sensitivity'], 
#                               nbinsx=20, histnorm='probability', name='(5, 10]'))
#     fig.add_trace(go.Histogram(x=df_metric[df_metric['rnafold_count'] > 10][metric_name], 
#                               nbinsx=20, histnorm='probability', name='>10'))

#     fig = fig.update(layout=dict(
#         paper_bgcolor='rgba(0,0,0,0)',
#         plot_bgcolor='rgba(0,0,0,0)',
#         xaxis=dict(showgrid=False, zeroline=False,
#                    tickfont=dict(size=18, color='#000'),
#                    titlefont=dict(size=24, color='#000'),
#                    showline=True, mirror=True),
#         yaxis=dict(showgrid=True, gridwidth=1, gridcolor='darkgray',
#                    tickfont=dict(size=18, color='#000'),
#                    titlefont=dict(size=24, color='#000'),
#                    title='Percentage', showline=True, mirror=True),
#     #     title='Performance on test dataset',
#         title=metric_name,
#         titlefont=dict(size=24, color='#000'),
#     #     legend=dict(orientation="h",
#     #                 font=dict(
#     # #             family='sans-serif',
#     #             size=24,
#     #             color='#000'
#     #         ),),
#     ))
#     iplot(fig)

In [53]:
def make_hist_2(df_plot, title):
#     fig = df_plot[['sensitivity', 'ppv']].iplot(kind='histogram', bins=20,
#                                                     histnorm='probability', opacity=1,
#                                                      barmode='group', asFigure=True)
#     fig = df_plot[['sensitivity', 'ppv']].iplot(kind='histogram', xbins=dict(
#                                                                     start=0.0,
#                                                                     end=1.0,
#                                                                     size=0.05
#                                                                 ),
#                                                     histnorm='probability', opacity=1,
#                                                      barmode='group', asFigure=True)
    fig = go.Figure()
    fig.add_trace(go.Histogram(
    x=df_plot['sensitivity'],
    histnorm='percent',
    name='Sensitivity', # name used in legend and hover labels
    xbins=dict( # bins used for histogram
        start=0.0,
        end=1.0,
        size=0.05
    ),
    marker_color='#EB89B5',
    opacity=0.75
    ))
    fig.add_trace(go.Histogram(
    x=df_plot['ppv'],
    histnorm='percent',
    name='PPV',
    xbins=dict( # bins used for histogram
        start=0.0,
        end=1.0,
        size=0.05
    ),
    marker_color='#330C73',
    opacity=0.75
    ))

    fig = fig.update(layout=dict(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    xaxis=dict(showgrid=False, zeroline=False,
               tickfont=dict(size=18, color='#000'),
               titlefont=dict(size=24, color='#000'),
               showline=True, mirror=True,
              range=[0,1]),
    yaxis=dict(showgrid=True, gridwidth=1, gridcolor='darkgray',
               tickfont=dict(size=18, color='#000'),
               titlefont=dict(size=24, color='#000'),
               title='Percentage', showline=True, mirror=True),
    title=title,
    titlefont=dict(size=24, color='#000'),
    legend=dict(orientation="h",
                font=dict(
            size=24,
            color='#000'
        ),),
    ))
    return fig

In [54]:
fig = make_hist_2(df_metric[(df_metric['rnafold_count']>=5) & (df_metric['len'] <= 100)],
                 'count>=5 & len<=100')
iplot(fig)

In [55]:
fig = make_hist_2(df_metric[(df_metric['rnafold_count']>=5) & (df_metric['len'] > 100)],
                 'count>=5 & len>100')
iplot(fig)

In [ ]:
# # uncomment to show plot in new window
# plot(fig)

In [ ]:
len(df_metric)

In [ ]:
sum(df_metric['ppv']>0.8)/float(len(df_metric))

In [ ]:
# # for R plot
# df_metric[['seq', 'sensitivity', 'ppv', 'f_measure']].to_csv('/Users/alicegao/data/tmp/perf_rand_ensemble_metric.csv', index=False)

In [ ]:
# # scatter plot

# fig = df_metric[['sensitivity', 'ppv']].iplot(kind='scatter', x='sensitivity', y='ppv',
#                                                     mode='markers', size=4,
#                                               asFigure=True)
# fig = fig.update(layout=dict(
#     paper_bgcolor='rgba(0,0,0,0)',
#     plot_bgcolor='rgba(0,0,0,0)',
#     xaxis=dict(showgrid=False, title='Sensitivity', zeroline=False),
#     yaxis=dict(showgrid=False, title='PPV'),
#     title='Performance on test dataset',
# ))

In [ ]:
# iplot(fig)